In [ ]:
# Data uploader
import os
import boto3
from botocore.exceptions import NoCredentialsError

def upload_to_s3(file_name, bucket, object_name=None, aws_access_key_id=None, aws_secret_access_key=None, region_name=None):
    """
    Upload a file to an S3 bucket using put_object

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified, file_name is used
    :param aws_access_key_id: AWS access key ID
    :param aws_secret_access_key: AWS secret access key
    :param region_name: AWS region name
    :return: True if file was uploaded, else False
    """
    
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Create an S3 client
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name=region_name
    )

    # Upload the file
    with open(file_name, 'rb') as file_data:
        s3_client.put_object(Bucket=bucket, Key=object_name, Body=file_data)
    print(f"File {file_name} uploaded to {bucket}/{object_name}")
    return True



# Example usage
file_name = "<FILENAME>"  # Local file path
bucket_name = "<BUCKET_NAME>"
object_name = f"raw/{file_name.split('/')[-1]}" 
print(object_name)

aws_access_key_id = '<AWS_ACCESS_KEY>'
aws_secret_access_key = '<AWS_SECRET_KEY>'
region_name = 'AWS_REGION'  # e.g., 'us-west-1'
directory_path = '<DIR_PATH>'

upload_to_s3(directory_path, bucket_name, aws_access_key_id, aws_secret_access_key, region_name)


In [ ]:
import boto3
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

def upload_to_s3(file_name, bucket, object_name=None, aws_access_key_id=None, aws_secret_access_key=None, region_name=None):
    # Initialize S3 client
    s3_client = boto3.client('s3',
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key,
                             region_name=region_name)
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        print(f"File {file_name} uploaded to {bucket}/{object_name}")
        return True
    except Exception as e:
        print(f"Failed to upload {file_name} to {bucket}/{object_name}: {e}")
        return False

def upload_files_concurrently(directory, bucket, aws_access_key_id=None, aws_secret_access_key=None, region_name=None):
    # Generate list of files from the directory
    file_names = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]
    
    # Create a list to hold futures
    futures = []
    
    # Use ThreadPoolExecutor to upload files concurrently
    with ThreadPoolExecutor() as executor:
        for file_name in file_names:
            object_name = f"raw/{os.path.basename(file_name)}"  # S3 object name
            futures.append(executor.submit(upload_to_s3, file_name, bucket, object_name, aws_access_key_id, aws_secret_access_key, region_name))
        
        # Ensure all futures are completed
        for future in as_completed(futures):
            future.result()  # This will raise an exception if the function call failed

# Directory containing files to upload
directory_path = "<FILENAME>"
bucket_name = "<BUCKET_NAME>"
aws_access_key_id = '<AWS_ACCESS_KEY>'
aws_secret_access_key = '<AWS_SECRET_KEY>'
region_name = '<AWS_REGION>'  # e.g., 'us-west-1'

# Upload all files concurrently
upload_files_concurrently(directory_path, bucket_name, aws_access_key_id, aws_secret_access_key, region_name)
